In [1]:
!ls -l ~/data

total 4
drwxr-xr-x 3 itv005077 students 4096 Apr 22 12:28 trendytech


In [2]:
!mkdir ~/data/input

In [3]:
!ls -l ~/data

total 8
drwxr-xr-x 2 itv005077 students 4096 Apr 24 15:54 input
drwxr-xr-x 3 itv005077 students 4096 Apr 22 12:28 trendytech


In [4]:
!ls ~/data/input/linkedin_views.csv

/home/itv005077/data/input/linkedin_views.csv


In [20]:
!cat ~/data/input/linkedin_views.csv

1,Manasa,Sumit
2,Deepa,Sumit
3,Sumit,Manasa
4,Manasa,Deepa
5,Deepa,Manasa
6,Shilpy,Manasa


In [21]:
!hadoop fs -ls data

Found 1 items
-rw-r--r--   3 itv005077 supergroup      18322 2023-04-22 12:28 data/search_data.txt


In [22]:
!hadoop fs -copyFromLocal ~/data/input/linkedin_views.csv data

In [23]:
!hadoop fs -ls data

Found 2 items
-rw-r--r--   3 itv005077 supergroup         90 2023-04-24 16:05 data/linkedin_views.csv
-rw-r--r--   3 itv005077 supergroup      18322 2023-04-22 12:28 data/search_data.txt


### find how many times each person's profile is viewed 

In [11]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder \
    .master('yarn') \
    .appName('Linkedin-Views') \
    .getOrCreate()

In [13]:
spark

In [15]:
sc = spark.sparkContext

In [16]:
sc

<SparkContext master=yarn appName=Linkedin-Views>

In [24]:
file_read_rdd = sc.textFile('/user/itv005077/data/linkedin_views.csv')

In [25]:
file_read_rdd.take(5)

['1,Manasa,Sumit',
 '2,Deepa,Sumit',
 '3,Sumit,Manasa',
 '4,Manasa,Deepa',
 '5,Deepa,Manasa']

In [30]:
linkedin_rdd = file_read_rdd.map(lambda row : row.split(','))

In [31]:
linkedin_rdd.take(5)

[['1', 'Manasa', 'Sumit'],
 ['2', 'Deepa', 'Sumit'],
 ['3', 'Sumit', 'Manasa'],
 ['4', 'Manasa', 'Deepa'],
 ['5', 'Deepa', 'Manasa']]

In [32]:
profile_rdd = linkedin_rdd.map(lambda col : col[2])

In [33]:
profile_rdd.take(5)

['Sumit', 'Sumit', 'Manasa', 'Deepa', 'Manasa']

In [34]:
individual_view_rdd = profile_rdd.countByValue()

In [35]:
individual_view_rdd

defaultdict(int, {'Sumit': 2, 'Manasa': 3, 'Deepa': 1})

In [37]:
for profile, no_views in individual_view_rdd.items():
    print(f'{profile}\'s profile has been viewed {no_views} times')

Sumit's profile has been viewed 2 times
Manasa's profile has been viewed 3 times
Deepa's profile has been viewed 1 times


### alternate approach

In [38]:
individual_view_rdd1 = profile_rdd.map(lambda profile : (profile,1))

In [39]:
individual_view_rdd1.take(5)

[('Sumit', 1), ('Sumit', 1), ('Manasa', 1), ('Deepa', 1), ('Manasa', 1)]

In [40]:
views_per_profile_rdd = individual_view_rdd1.reduceByKey(lambda x,y : x+y)

In [41]:
views_per_profile_rdd.take(5)

[('Sumit', 2), ('Deepa', 1), ('Manasa', 3)]

In [42]:
views = views_per_profile_rdd.collect()

In [43]:
views

[('Sumit', 2), ('Deepa', 1), ('Manasa', 3)]

In [44]:
for profile, no_views in views:
    print(f'{profile}\'s profile has been viewed {no_views} times')

Sumit's profile has been viewed 2 times
Deepa's profile has been viewed 1 times
Manasa's profile has been viewed 3 times
